# First Things First

To start with, I'm simply going to build a basic web crawler then, using the domains from the URDB source URLs file, I'll randomly sample 1000 to see what their robots.txt file says (to keep it simple, I'll parse them first to see which ones are empty/missing, then check to see what the non-missing/non-empty ones say. For reference, the standards for constructing a robots.txt file [is explained here](http://www.robotstxt.org/robotstxt.html).

In [1]:
import pandas as pd

In [2]:
URLs = pd.read_table('URDB_URLs_9-24-18.txt', names = ['Tariff Sheet URL'])
URLs.drop_duplicates(inplace = True)
URLs

,Tariff Sheet URL
0,http://www.iid.com/Modules/ShowDocument.aspx?d...
1,http://villageofarcade.org/departments/public-...
2,http://cecelec.coopwebbuilder.com/sites/cecele...
3,http://www.wakeforestnc.gov/client_resources/r...
4,http://www.rrvcoop.com/content/security-lights
5,http://www.prairielandelectric.com/Rates_PDF/M...
6,http://www.talquinelectric.com/rates_elec.aspx
7,http://www.midstateelectric.coop/customer-serv...
8,http://www.surprisevalleyelectric.org/_uploads...
9,http://www.mainepublicservice.com/media/40402/...


# The Procedure

Based upon some very brief looking-over of these data, it seems like I'll need to do a few things to clean these URLs up first:

1. Check how many unique values there are relative to the DataFrame size
2. Reduce all URLs to their primary domain (e.g. https://www.nyseg.com/)
3. Remove all duplicates
4. Slap "robots.txt" on the end
5. Drop duplicate URLs again, since we probably now have more!
5. Run these through scrapy

In [3]:
#Clean up some messy URLs - na = True because we intend to apply ~ to NOT the resultant series ultimately
is_http = (URLs['Tariff Sheet URL'].str.split("/", expand = True)[0].str.contains("http", na = True))
is_ftp = (URLs['Tariff Sheet URL'].str.split("/", expand = True)[0].str.contains("ftp", na = True))

bad_URLs = (~is_http & ~is_ftp)
bad_URLs.sum()

URLs.loc[bad_URLs,'Tariff Sheet URL'] = "h" + URLs.loc[bad_URLs,'Tariff Sheet URL']
#URLs.loc[36010]

In [4]:
#Find only the domain portion of the URL + http... and then add /robots.txt to the end

#No good, it doesn't work for FTP sites
URLs['Utility Domain URL'] = URLs['Tariff Sheet URL'].str.extract(pat = r'(https*://[^/]+)')

URLs['robots.txt URL'] = URLs['Utility Domain URL'] + '/robots.txt'

URLs.drop_duplicates(inplace = True)
URLs

,Tariff Sheet URL,Utility Domain URL,robots.txt URL
0,http://www.iid.com/Modules/ShowDocument.aspx?d...,http://www.iid.com,http://www.iid.com/robots.txt
1,http://villageofarcade.org/departments/public-...,http://villageofarcade.org,http://villageofarcade.org/robots.txt
2,http://cecelec.coopwebbuilder.com/sites/cecele...,http://cecelec.coopwebbuilder.com,http://cecelec.coopwebbuilder.com/robots.txt
3,http://www.wakeforestnc.gov/client_resources/r...,http://www.wakeforestnc.gov,http://www.wakeforestnc.gov/robots.txt
4,http://www.rrvcoop.com/content/security-lights,http://www.rrvcoop.com,http://www.rrvcoop.com/robots.txt
5,http://www.prairielandelectric.com/Rates_PDF/M...,http://www.prairielandelectric.com,http://www.prairielandelectric.com/robots.txt
6,http://www.talquinelectric.com/rates_elec.aspx,http://www.talquinelectric.com,http://www.talquinelectric.com/robots.txt
7,http://www.midstateelectric.coop/customer-serv...,http://www.midstateelectric.coop,http://www.midstateelectric.coop/robots.txt
8,http://www.surprisevalleyelectric.org/_uploads...,http://www.surprisevalleyelectric.org,http://www.surprisevalleyelectric.org/robots.txt
9,http://www.mainepublicservice.com/media/40402/...,http://www.mainepublicservice.com,http://www.mainepublicservice.com/robots.txt


In [5]:
#All split results are 3-element lists of str, with [2] being the bit we care about
#Since I can't do Series.str.split("/", expand = True) for some odd reason, this will have to do
URLs['Utility Domain (WWW Only)'] = URLs['Utility Domain URL'].str.split("/").str.get(-1)
URLs['Utility Domain (WWW Only)'].drop_duplicates()

0                                 www.iid.com
1                         villageofarcade.org
2                  cecelec.coopwebbuilder.com
3                        www.wakeforestnc.gov
4                             www.rrvcoop.com
5                 www.prairielandelectric.com
6                     www.talquinelectric.com
7                   www.midstateelectric.coop
8              www.surprisevalleyelectric.org
9                  www.mainepublicservice.com
10                            www.ecirec.coop
11                                 psc.wi.gov
12                       www.singingriver.com
13                          www.delaware.coop
15                        www.bartlettec.coop
16                      www.clatskaniepud.com
17                www.sanpatricioelectric.org
18                               www.npec.org
19                         www.neelectric.com
20                                sfwater.org
21                             www.brmemc.com
22                               w

In [6]:
URLs.isnull().sum()

Tariff Sheet URL             0
Utility Domain URL           2
robots.txt URL               2
Utility Domain (WWW Only)    2
dtype: int64

In [7]:
#URLs['Tariff Sheet URL'].str.split(pat = r'^https*://.+/', expand = True)
#This doesn't do what I want right now, but gets the filenames, so maybe useful down the road?

In [8]:
URLs['Utility Domain URL'].nunique()

1979

In [9]:
URLs['Crawling_Allowed'] = False

In [10]:
URLs['Num_Ratepayers'] = 0

In [11]:
URLs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7920 entries, 0 to 39042
Data columns (total 6 columns):
Tariff Sheet URL             7920 non-null object
Utility Domain URL           7918 non-null object
robots.txt URL               7918 non-null object
Utility Domain (WWW Only)    7918 non-null object
Crawling_Allowed             7920 non-null bool
Num_Ratepayers               7920 non-null int64
dtypes: bool(1), int64(1), object(4)
memory usage: 379.0+ KB


In [12]:
#Export as CSV file for pulling into scrapy spider
#Only need the domain URLs for robots checking, but push out all of them anyhow
URLs.to_csv("Rate_URLs.csv")

In [13]:
#Need to parse log for robots data that allowed crawling
lines = []

with open('URDB/logs/utility_10-23-18.log','r') as f:
    for line in f:
        #Count the good crawls
        if "DEBUG: Crawled" in line:
            lines.append(line)
            
print(lines[:5])

['2018-10-23 22:23:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.iid.com/robots.txt> (referer: None)\n', '2018-10-23 22:23:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.prairielandelectric.com/robots.txt> (referer: None)\n', '2018-10-23 22:23:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://www.midstateelectric.coop/robots.txt> (referer: None)\n', '2018-10-23 22:23:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://villageofarcade.org/robots.txt> (referer: None)\n', '2018-10-23 22:23:38 [scrapy.core.engine] DEBUG: Crawled (404) <GET http://www.mainepublicservice.com/robots.txt> (referer: None)\n']


In [14]:
crawled = pd.Series(lines)
crawled = crawled.str.split('DEBUG: Crawled', expand = True)[1]

#Only gives domains for extracts that have robots.txt in the log line, as other lines report on the actual crawl
    #MAY be conservative in that there may not be a successful crawl log line for domain missing robots.txt
crawled = crawled.str.extract(r'<GET https*://(.*)/robots.txt>').dropna()

crawled = crawled[0]
crawled

0                         www.iid.com
1         www.prairielandelectric.com
2           www.midstateelectric.coop
3                 villageofarcade.org
4          www.mainepublicservice.com
5      www.surprisevalleyelectric.org
6             www.talquinelectric.com
8                     www.rrvcoop.com
9                www.wakeforestnc.gov
13                   singingriver.com
15                    www.ecirec.coop
16                www.bartlettec.coop
18                   singingriver.com
21                         psc.wi.gov
22              www.clatskaniepud.com
23                  www.delaware.coop
24                 www.neelectric.com
30        www.sanpatricioelectric.org
31                     www.brmemc.com
32                       www.cme.coop
37                   www.iplpower.com
38                      www.iclp.coop
39         www.claycountyelectric.com
42        www.morristownutilities.org
43          www.adrian.govoffice2.com
44                     www.swepco.com
45          

In [15]:
#Fuzzy match to existing WWW-only domains
from TextSimilarity import best_option

best_option("vec.org", crawled, cluster = True, score_threshold = 0.75)

('vec.org', nan, 75)

# TO DO
1. Pull scrapy log data into a DataFrame for more thorough cleaning/parsing
    1. Collect lines that contain "DEBUG: Crawled" into a list of str
    2. Push that list into a DataFrame and split the results on " DEBUG: Crawled (200) ", discarding `[0]` in favor of `[1]`
    3. From `[1]` get URL
        * URL = regex find on `"<GET ...url...>"`
    3. Perform fuzzy matching on existing domains to make sure we correctly associate with all possible matched to get a semi-reasonable % matched
    4. Associate final URL (no http or https) with `[Utility Domain (WWW Only)]` unduplicated entries, setting `['Crawling_Allowed'] = True` for those. Then figure out what fraction of the total number of nonduplicated domains have successful crawls
2. Come up with a more explicit way of determining if robots.txt is blocking us from rate pages
    * Actually try crawling the tariff sheet URL we have on file, but only one per domain to speed things up, even if just a rough estimate
    * Figure out from this what a blocking robots.txt looks like in the logs, keeping in mind that redirects may simply be going from `http` to `https` or something similar that isn't a true block
3. For those with `Crawling_Allowed = True`, use their `Num_Ratepayers` value their score and final metric of "% Allowed" will be `(URLs['Num_Ratepayers'] * URLs['Crawling_Allowed']).sum()/URLs['Num_Ratepayers'].sum()`